In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Carga de informacion accesoria

In [2]:
import pandas as pd

In [3]:
# Adulto equivalente. Cuanto cuesta la manutencion de las personas segun sexo y edad.
ad_eq = pd.read_csv('./../data/info/adulto_eq.csv')

#Importar canasta basica regional deflac
CB_ipc = pd.read_csv('./../data/info/CB_Reg_defl.csv')
# ppp_defl = pd.read_csv('./../data/info/ppp_defl.csv')

# Load radio ref. Merge regiones.
# Anything that is AGLOMERADO 33 should be region Gran Buenos Aires

# radio_ref = pd.read_csv('./../data/info/radio_ref.csv').merge(pd.read_csv('./../data/info/prov_regs.csv'), how = 'left')

radio_ref = pd.read_csv('./../data/info/radio_ref.csv')#.merge(aglo_labels)
dpto_region = pd.read_csv('./../data/info/DPTO_PROV_Region.csv')
## Fix temporario, migracion a nombres de region oficiales
dpto_region['Region'] = dpto_region['Region'].map({'Gran Buenos Aires':'gran_buenos_aires', 
                                                   'Pampeana':'pampeana', 'Noroeste':'noroeste', 
                                                   'Noreste':'noreste','Patagónica': 'patagonia', 'Cuyo': 'cuyo'})

radio_ref = radio_ref.merge(dpto_region)

# DPTO_Region = radio_ref[['DPTO', 'Region']].drop_duplicates()

# dpto_region = pd.read_csv('./../data/info/DPTO_PROV_Region.csv')
DPTO_Region = radio_ref[['DPTO', 'Region']].drop_duplicates()

In [4]:
import os
if not os.path.exists('./../data/Pobreza/'):
    os.makedirs('./../data/Pobreza/')

frac = 0.01

## Funcion Generar dataset Pobreza

In [5]:

def ingresos_a_pobreza(df_ingresos, filename, columnas_pesos = ['P47T']):

    df_ingresos[columnas_pesos] = np.power(10, df_ingresos[columnas_pesos]) - 1
    
    # Editar columnas:
    ## Nivel Educativo
    df_ingresos['P10'] = 2 - df_ingresos['P10']  ## Re codificacion de pregunta 'termino el nivel'
    df_ingresos['P09'] = df_ingresos.P09.replace(5, 4) # Polimodal tomado como secundario 
    df_ingresos['P0910'] = df_ingresos.P09.astype(str) + df_ingresos.P10.astype(str)
    
    ## Grupos Etarios
    df_ingresos['Grupo_Etario_3'] = pd.cut(df_ingresos.P03, np.arange(-1, 80, 3)).astype(str)#.round(-1)
    df_ingresos['Grupo_Etario_INDEC'] = pd.cut(df_ingresos.P03, np.array([0, 13, 29, 64, 110])).astype(str)#.round(-1)
    df_ingresos['Grupo_Etario_q10'] = pd.cut(df_ingresos.P03, np.array([-0.001, 5.0,  11.0, 17.0, 23.0, 29.0, 36.0, 44.0, 53.0, 65.0, 110.0])).astype(str)#.round(-1)
    # df = pd.read_csv('file.csv', dtype={'Col' : 'category'}) # Despues podemos necesitar...

    df = df_ingresos.reset_index()

    ## CANASTA: Datos mergeado con adulto equivalente, region y serie de tiempo canasta
    df_cb = df_ingresos.merge(ad_eq).merge(DPTO_Region).merge(CB_ipc)#.merge(ppp_defl[['Q', 'ppp_5usd_ARS_deflac']])
    df_cb['CBA'] = df_cb['CBA']*df_cb['CB_EQUIV']  ## Con este paso el valor de canasta de una persona YA INCORPORA EL AD EQ
    df_cb['CBT'] = df_cb['CBT']*df_cb['CB_EQUIV']  ## Con este paso el valor de canasta de una persona YA INCORPORA EL AD EQ

    ## VARIABLES A NIVEL HOGARES
#     df_cb_hogares = df_cb.groupby(['HOGAR_REF_ID', 'Q'])[['P47T','CBA', 'CBT', 'CB_EQUIV', 'ppp_5usd_ARS_deflac']].sum()
    df_cb_hogares = df_cb.groupby(['HOGAR_REF_ID', 'Q'])[['P47T','CBA', 'CBT', 'CB_EQUIV']].sum()
    df_cb_hogares['Pobreza'] = df_cb_hogares['P47T'] < df_cb_hogares['CBT']
    df_cb_hogares['Indigencia'] = df_cb_hogares['P47T'] < df_cb_hogares['CBA']
#     df_cb_hogares['Pobreza_5usd'] = df_cb_hogares['P47T'] < df_cb_hogares['ppp_5usd_ARS_deflac']
#     pobreza_hogares = df_cb_hogares[['P47T','CBA','CBT', 'CB_EQUIV','Pobreza', 'Indigencia', 'Pobreza_5usd']].reset_index()
    pobreza_hogares = df_cb_hogares[['P47T','CBA','CBT', 'CB_EQUIV','Pobreza', 'Indigencia']].reset_index()
    pobreza_hogares['gap_pobreza'] = pobreza_hogares.P47T - pobreza_hogares.CBT
    pobreza_hogares['gap_indigencia'] = pobreza_hogares.P47T - pobreza_hogares.CBA
    pobreza_hogares = pobreza_hogares.rename(columns = {'P47T': 'P47T_hogar'})

    ## UNION DE DATOS DE HOGARES A REGISTROS INDIVIDUALES
    data = df_ingresos.merge(pobreza_hogares, on = ['HOGAR_REF_ID', 'Q'])#, how = 'left')
    del df; del pobreza_hogares # Ahorrar memoria
    data = data.rename(columns = {'P47T': 'P47T_persona'}) # Renombrar la variable P47T para aclarar que es a nivel persona.

    ## UNIR INFO GEOGRAFICA
    data = data.merge(radio_ref[['RADIO_REF_ID', 'IDFRAC', 'PROV', 'NOMPROV', 'AGLOMERADO', 'Region']].drop_duplicates())

    n_q = data.Q.nunique()
    print("Poblacion: "+str(len(data)/frac/n_q))
#     display(data[['Pobreza', 'Indigencia', 'Pobreza_5usd']].mean())
    data.to_csv(filename, index = False) ## Aca si ya existen no deberian sobreescribirse (o si)
    
    print(filename+' saved')

## Lista de archivos

In [6]:
import glob
from pathlib import Path
import numpy as np

# path ='./data/RFReg_' # use your path
# path ='./../../encuestador-de-hogares/data/yr_samples/RFReg_' # use your path
path ='/media/miglesia/Elements/suite/yr_samples/RFReg_'

allFiles = []
for year in [str(s) for s in range(2003, 2024)]:
    allFiles += glob.glob(path +str(frac)+ '*'+str(year)+'*_ARG.csv')
    # Estos son los archivos que se usan para tener una figura estatica, corte donde no importa evol. temporal.

allFiles = sorted(allFiles) # ultimo anio
allFiles

['/media/miglesia/Elements/suite/yr_samples/RFReg_0.01_2003-08-15_ARG.csv',
 '/media/miglesia/Elements/suite/yr_samples/RFReg_0.01_2003-11-15_ARG.csv',
 '/media/miglesia/Elements/suite/yr_samples/RFReg_0.01_2004-02-15_ARG.csv',
 '/media/miglesia/Elements/suite/yr_samples/RFReg_0.01_2004-05-15_ARG.csv',
 '/media/miglesia/Elements/suite/yr_samples/RFReg_0.01_2004-08-15_ARG.csv',
 '/media/miglesia/Elements/suite/yr_samples/RFReg_0.01_2004-11-15_ARG.csv',
 '/media/miglesia/Elements/suite/yr_samples/RFReg_0.01_2005-02-15_ARG.csv',
 '/media/miglesia/Elements/suite/yr_samples/RFReg_0.01_2005-05-15_ARG.csv',
 '/media/miglesia/Elements/suite/yr_samples/RFReg_0.01_2005-08-15_ARG.csv',
 '/media/miglesia/Elements/suite/yr_samples/RFReg_0.01_2005-11-15_ARG.csv',
 '/media/miglesia/Elements/suite/yr_samples/RFReg_0.01_2006-02-15_ARG.csv',
 '/media/miglesia/Elements/suite/yr_samples/RFReg_0.01_2006-05-15_ARG.csv',
 '/media/miglesia/Elements/suite/yr_samples/RFReg_0.01_2006-08-15_ARG.csv',
 '/media/mig

## 1.  Resultados estaticos (se toma 1 año)

In [7]:
# years = np.unique([int(f[-14:-10]) for f in  allFiles])

recentFiles = allFiles[-4:]
qstrings = np.unique([Path(f).name.split('_')[-2] for f in  recentFiles[-4:]])
ystrings = np.unique([Path(f).name.split('_')[-2].split('-')[0] for f in  recentFiles[-4:]])
years = [int(y) for y in ystrings]
years
if len(years) == 1:
    yr_label = str(years[0])
else:
    yr_label = '-'.join([str(years[0]), str(years[-1])])
    
yr_label

'2021-2022'

In [8]:
### Dataset Ultimo anio
df_parts = []
for quarter_Xy_file in sorted(recentFiles):# ultimo anio
    df_Q = pd.read_csv(quarter_Xy_file, 
                           usecols = ['PERSONA_REF_ID', 'HOGAR_REF_ID','RADIO_REF_ID', 'CONDACT', 'CAT_INAC', 'CAT_OCUP',
                                      'INGRESO', 'INGRESO_NLB', 'INGRESO_JUB', 'INGRESO_SBS', 'PP07K',
                                      'IX_TOT', 'H16', 'H15','P47T', 'P03','P02', 'P09','P10', 'DPTO', 'URP'])
    
    df_Q['ANO4'] = int(Path(quarter_Xy_file).name.split('_')[-2].split('-')[0])
    q = Path(quarter_Xy_file).name.split('_')[-2]; print(q)
    df_Q['Q'] = q
    df_parts += [df_Q]

df = pd.concat(df_parts)
del df_Q


2021-05-15
2021-08-15
2021-11-15
2022-02-15


#### Computar Pobreza

In [9]:
filename = '/media/miglesia/Elements/suite/indice-pobreza-ExactasUBA/data/Pobreza/pobreza_'+'_'.join([str(frac), yr_label])+'.csv'

ingresos_a_pobreza(df_ingresos = df, filename = filename, columnas_pesos = ['P47T'])

Poblacion: 45383225.0
/media/miglesia/Elements/suite/indice-pobreza-ExactasUBA/data/Pobreza/pobreza_0.01_2021-2022.csv saved


## 2. Resultados series de tiempo (se computa para todos y cada trimestre)

In [11]:
for quarter_Xy_file in sorted(allFiles):
#     if not os.path.exists('./../data/Pobreza/pobreza_'+'_'.join([str(frac), 'q'+q])+'.csv'):

    df_Q = pd.read_csv(quarter_Xy_file, 
                           usecols = ['PERSONA_REF_ID', 'HOGAR_REF_ID','RADIO_REF_ID', 'CONDACT', 'CAT_INAC', 'CAT_OCUP',
                                      'INGRESO', 'INGRESO_NLB', 'INGRESO_JUB', 'INGRESO_SBS', 'PP07K',
                                      'IX_TOT', 'H16', 'H15','P47T', 'P03','P02', 'P09','P10', 'DPTO', 'URP'])
    df_Q['ANO4'] = int(Path(quarter_Xy_file).name.split('_')[-2].split('-')[0])
    q = Path(quarter_Xy_file).name.split('_')[-2]; print(q)
    df_Q['Q'] = q

    ## Computar pobreza
    ingresos_a_pobreza(df_ingresos = df_Q, filename = '/media/miglesia/Elements/suite/indice-pobreza-ExactasUBA/data/Pobreza/pobreza_'+'_'.join([str(frac), 'q'+q])+'.csv', columnas_pesos = ['P47T'])


2003-08-15
Poblacion: 36481600.0
/media/miglesia/Elements/suite/indice-pobreza-ExactasUBA/data/Pobreza/pobreza_0.01_q2003-08-15.csv saved
2003-11-15
Poblacion: 36481600.0
/media/miglesia/Elements/suite/indice-pobreza-ExactasUBA/data/Pobreza/pobreza_0.01_q2003-11-15.csv saved
2004-02-15
Poblacion: 37177100.0
/media/miglesia/Elements/suite/indice-pobreza-ExactasUBA/data/Pobreza/pobreza_0.01_q2004-02-15.csv saved
2004-05-15
Poblacion: 37177100.0
/media/miglesia/Elements/suite/indice-pobreza-ExactasUBA/data/Pobreza/pobreza_0.01_q2004-05-15.csv saved
2004-08-15
Poblacion: 37177100.0
/media/miglesia/Elements/suite/indice-pobreza-ExactasUBA/data/Pobreza/pobreza_0.01_q2004-08-15.csv saved
2004-11-15
Poblacion: 37177100.0
/media/miglesia/Elements/suite/indice-pobreza-ExactasUBA/data/Pobreza/pobreza_0.01_q2004-11-15.csv saved
2005-02-15
Poblacion: 16074400.0
/media/miglesia/Elements/suite/indice-pobreza-ExactasUBA/data/Pobreza/pobreza_0.01_q2005-02-15.csv saved
2005-05-15
Poblacion: 16074400.0
/

2019-08-15
Poblacion: 44137500.0
/media/miglesia/Elements/suite/indice-pobreza-ExactasUBA/data/Pobreza/pobreza_0.01_q2019-08-15.csv saved
2019-11-15
Poblacion: 44137500.0
/media/miglesia/Elements/suite/indice-pobreza-ExactasUBA/data/Pobreza/pobreza_0.01_q2019-11-15.csv saved
2020-02-15
Poblacion: 44615600.0
/media/miglesia/Elements/suite/indice-pobreza-ExactasUBA/data/Pobreza/pobreza_0.01_q2020-02-15.csv saved
2020-05-15
Poblacion: 44615600.0
/media/miglesia/Elements/suite/indice-pobreza-ExactasUBA/data/Pobreza/pobreza_0.01_q2020-05-15.csv saved
2020-08-15
Poblacion: 44615600.0
/media/miglesia/Elements/suite/indice-pobreza-ExactasUBA/data/Pobreza/pobreza_0.01_q2020-08-15.csv saved
2020-11-15
Poblacion: 44615600.0
/media/miglesia/Elements/suite/indice-pobreza-ExactasUBA/data/Pobreza/pobreza_0.01_q2020-11-15.csv saved
2021-02-15
Poblacion: 45325100.0
/media/miglesia/Elements/suite/indice-pobreza-ExactasUBA/data/Pobreza/pobreza_0.01_q2021-02-15.csv saved
2021-05-15
Poblacion: 45325100.0
/

## 3. Resultados semestre

In [12]:
files =  ['/media/miglesia/Elements/suite/yr_samples/RFReg_0.01_2021-08-15_ARG.csv',
 '/media/miglesia/Elements/suite/yr_samples/RFReg_0.01_2021-11-15_ARG.csv']

In [13]:
### Dataset Ultimo anio
df_parts = []
for quarter_Xy_file in sorted(files):# ultimo anio
    df_Q = pd.read_csv(quarter_Xy_file, 
                           usecols = ['PERSONA_REF_ID', 'HOGAR_REF_ID','RADIO_REF_ID', 'CONDACT', 'CAT_INAC', 'CAT_OCUP',
                                      'INGRESO', 'INGRESO_NLB', 'INGRESO_JUB', 'INGRESO_SBS', 'PP07K',
                                      'IX_TOT', 'H16', 'H15','P47T', 'P03','P02', 'P09','P10', 'DPTO', 'URP'])
    
    df_Q['ANO4'] = int(Path(quarter_Xy_file).name.split('_')[-2].split('-')[0])
    q = Path(quarter_Xy_file).name.split('_')[-2]; print(q)
    df_Q['Q'] = q
    df_parts += [df_Q]

df = pd.concat(df_parts)
del df_Q


2021-08-15
2021-11-15


In [14]:
label = '20212'
filename = '/media/miglesia/Elements/suite/indice-pobreza-ExactasUBA/data/Pobreza/pobreza_'+'_'.join([str(frac), label])+'.csv'

ingresos_a_pobreza(df_ingresos = df, filename = filename, columnas_pesos = ['P47T'])

Poblacion: 45325100.0
/media/miglesia/Elements/suite/indice-pobreza-ExactasUBA/data/Pobreza/pobreza_0.01_20212.csv saved


In [ ]:
df.shape

In [ ]:
len(df)/frac/2